# Valuing Options on Commodity Futures

The Black-Scholes equation is the well known model to price equity European options. In the case of equities, the spot price fluctuates and hence the intuition to model as a stochastic process makes sense. In the case of commodities, however, the spot price does not fluctuate as much, and hence cannot be modeled as a stochastic process to value options on commodities. In the 1976 paper [1], Fischer Black addressed this problem by modeling the futures price, which demonstrates fluctuations, as the lognormal stochastic process.

The futures price at time $t, F_t$ with a is modeled as:

$dF_t = \sigma_t F_t dW_t$

where $\sigma_t$ is the volatility of the underlying, and $dW$ is the Weiner process.

The value of an option at strike $K$, maturing at $T$, risk free rate $r$ with volatility $\sigma$ of the underlying is given by the closed form expression:

$c = e^{-rT} [ F\cdot N(d_1) - K\cdot N(d_2) ]$
$p = e^{-rT} [ K\cdot N(-d_2) - F\cdot N(-d_1)]$

where

$d_1 = \frac{ln(F/K) + (\sigma^2/2)T}{\sigma \sqrt{T}}$
$d_2 = \frac{ln(F/K) - (\sigma^2/2)T}{\sigma \sqrt{T}} = d_1 - \sigma \sqrt{T}$


This formula can be easily used to price **caps, swaptions, futures options contract**. Here we will use QuantLib to price the options on commodity futures.

In [7]:
import QuantLib as ql
import math

calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
bussiness_convention = ql.ModifiedFollowing
settlement_days = 0
day_count = ql.ActualActual(ql.ActualActual.ISMA)

In [8]:
interest_rate = 0.0015
calc_date = ql.Date(23,9,2015)
yield_curve = ql.FlatForward(calc_date, 
                             interest_rate,
                             day_count,
                             ql.Compounded,
                             ql.Continuous)

In [9]:
ql.Settings.instance().evaluationDate = calc_date
T = 96.12/365.

strike = 3.5
spot = 2.919
volatility = 0.4251
flavor = ql.Option.Call

discount = yield_curve.discount(T)
strikepayoff = ql.PlainVanillaPayoff(flavor, strike)
stddev = volatility*math.sqrt(T)

strikepayoff = ql.PlainVanillaPayoff(flavor, strike)
black = ql.BlackCalculator(strikepayoff, spot, stddev, discount)

In [10]:
print("%-20s: %4.4f" %("Option Price", black.value() ))
print("%-20s: %4.4f" %("Delta", black.delta(spot) ))
print("%-20s: %4.4f" %("Gamma", black.gamma(spot) ))
print("%-20s: %4.4f" %("Theta", black.theta(spot, T) ))
print("%-20s: %4.4f" %("Vega", black.vega(T) ))
print("%-20s: %4.4f" %("Rho", black.rho( T) ))

Option Price        : 0.0789
Delta               : 0.2347
Gamma               : 0.4822
Theta               : -0.3711
Vega                : 0.4600
Rho                 : 0.1597
